<a href="https://colab.research.google.com/github/Ahmetyldrr/PyTorch-Ultimate/blob/main/Multi_Label_Classification_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# %% Gerekli paketlerin import edilmesi

## CHAPGPT
#--------------------------------------
# - sklearn'den make_multilabel_classification: Çok etiketli sınıflandırma için sentetik veri oluşturur.
# - train_test_split: Veriyi eğitim ve test olarak böler.
# - accuracy_score: Sınıflandırma doğruluk skorunu hesaplar.
# - PyTorch'un temel bileşenleri: tensör, modül (nn), veri yüklemek için DataLoader, Dataset vb.
# - seaborn ve collections: Veri görselleştirme (kaynak kodun sonuna doğru scatterplot) ve sayım işlemleri için kullanılır.

from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import seaborn as sns
import numpy as np
from collections import Counter


# %% Veri oluşturma (data prep)
#--------------------------------
# make_multilabel_classification fonksiyonu ile sahte (synthetic) çok etiketli bir veri kümesi oluşturuyoruz.
# Bu veri kümesinde:
#   - n_samples=100: 100 örnek (örnek sayısı)
#   - n_features=10: 10 öz nitelik (feature)
#   - n_classes=3: 3 farklı etiket
#   - n_labels=2: Her örnekte ortalama 2 etiketin aktif olması beklenir.
# Üretilen X, y ikililerinde X, girdi özelliklerini; y ise (örneğin [1, 0, 1] gibi) etiket vektörlerini barındırır.

X, y = make_multilabel_classification(n_samples=100,
                                      n_features=10,
                                      n_classes=3,
                                      n_labels=2)

# Üretilen numpy dizilerini PyTorch tensörlerine dönüştürüyoruz.
X_torch = torch.FloatTensor(X)
y_torch = torch.FloatTensor(y)

# Veriyi eğitim ve test olarak bölerken, test boyutu %20 olarak ayarlanır.
# X_train, X_test: Özellikler
# y_train, y_test: Etiketler
X_train, X_test, y_train, y_test = train_test_split(X_torch,
                                                    y_torch,
                                                    test_size=0.2)


# %% Dataset ve DataLoader tanımlanması
#---------------------------------------
# PyTorch'ta kendi veri setimizi, Dataset sınıfından türeterek tanımlıyoruz.
# Bu şekilde DataLoader ile verileri kolayca kümelere (batch) ayırabilir,
# eğitim döngüsünde bu verileri kullanabiliriz.

class MultilabelDataset(Dataset):
    def __init__(self, X, y):
        # X ve y, yapıcı fonksiyonda (constructor) saklanıyor
        self.X = X
        self.y = y

    def __len__(self):
        # Bu fonksiyon, veri setinin kaç örnek (sample) içerdiğini döndürür
        return len(self.X)

    def __getitem__(self, idx):
        # Bu fonksiyon, belirli bir index’e (idx) sahip tek bir örneğin
        # özelliklerini (X) ve etiketlerini (y) döndürür
        return self.X[idx], self.y[idx]


# DataLoader: Verileri batch halinde almak, shuffle yapmak ve
# çok daha kolay veri yönetimi sağlamak amacıyla kullanılır.
multilabel_data = MultilabelDataset(X_train, y_train)
train_loader = DataLoader(dataset = multilabel_data, batch_size=10)
# batch_size=10 demek, her seferinde 10 örneklik mini-batch'ler ile eğitime girileceği anlamına gelir.


# %% Model tanımlaması
#---------------------------------------
# Basit bir Tam Bağlantılı (Fully Connected) sinir ağı tanımlıyoruz (MultilabelNetwork).
# Ağı yapısı:
# - fc1: Girdi boyutunu gizli katmana çeviren tam bağlantılı katman
# - ReLU: Aktivasyon fonksiyonu
# - fc2: Gizli katmandan çıkış katmanına
# - Sigmoid: Çıkışta [0,1] arası değerler elde etmek için kullanıyoruz.
# Çok etiketli sınıflandırmada, her bir etiketin aktif olup olmama olasılığını üretir.

class MultilabelNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MultilabelNetwork, self).__init__()
        # 1. Katman: input_size -> hidden_size
        self.fc1 = nn.Linear(input_size, hidden_size)
        # Aktivasyon: ReLU
        self.relu = nn.ReLU()
        # 2. Katman: hidden_size -> output_size
        self.fc2 = nn.Linear(hidden_size, output_size)
        # Çıkışı [0,1] aralığına map eden sigmoid aktivasyon
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # İleri yönlü geçiş (forward pass)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        # Çok etiketli tahmin için sigmoid ile skorları 0-1 arasına sıkıştırıyoruz
        x = self.sigmoid(x)
        return x

# Girdi boyutu (input_dim) ve çıktı boyutu (output_dim) veriyi okuyarak bulunur.
input_dim = X_torch.shape[1]   # X'in sütun sayısı -> 10
output_dim = y_torch.shape[1]  # y'nin sütun sayısı -> 3

# Modeli belirtilen boyutlarla örnekliyoruz:
# - input_size=10, hidden_size=20, output_size=3
model = MultilabelNetwork(input_size=input_dim,
                          hidden_size=20,
                          output_size=output_dim)

# model.train() metodu, modülü eğitim moduna alır. (Dropout, BatchNorm gibi katmanlar varsa etkili olur)
model.train()


# %% Kayıp fonksiyonu (Loss) ve Optimizasyon (Optimizer) seçimi
#---------------------------------------------------------------
# - BCEWithLogitsLoss: Çoklu etiketli sınıflandırmada sıkça kullanılan
#   binary cross-entropy tabanlı bir kayıp fonksiyonudur.
# - Adam: Popüler bir optimizasyon algoritmasıdır (momentum, adaptif öğrenme oranı vb. özellikler barındırır).
# (Not: BCEWithLogitsLoss, modelin çıktılarına genelde sigmoid uygulaması entegre eder;
#  fakat yukarıdaki gibi model içinde sigmoid kullandığınızda da BCE'yi kullanabilirsiniz.
#  Tek fark 'WithLogits' öncesi tipik kullanımda sigmoid modeli kayıp fonksiyonu içinde olur.)

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# eğitim süresince takip etmek için kayıpları (loss) bir listede saklıyoruz
losses = []
# Aşağıda basit bir şekilde epoch sayısını 100 olarak belirledik.
number_epochs = 100

# Eğitim döngüsü
for epoch in range(number_epochs):
    for j, data in enumerate(train_loader):
        # train_loader bize her seferinde 10'luk mini-batch (X, y) getiriyor
        # data[0]: X (özellikler), data[1]: y (etiketler)

        # 1. Adım: Geriye dönük hesaplamaları (gradient) sıfırlama
        optimizer.zero_grad()

        # 2. Adım: İleri geçiş (forward)
        y_hat = model(data[0])

        # 3. Adım: Kayıp (loss) hesaplama
        # y_hat: modelin tahminleri
        # data[1]: gerçek etiket değerleri
        loss = loss_fn(y_hat, data[1])
        losses.append(loss.item())

        # 4. Adım: Geriye yayılım (backpropagation)
        loss.backward()

        # 5. Adım: Ağırlık güncelleme
        optimizer.step()

    # Belirli epoch'larda durum bilgisi yazdırma
    if (epoch % 10 == 0):
        print(f"Epoch {epoch}, Loss: {loss.data}")


# %% Kayıpları (Loss) görselleştirme
#-------------------------------------
# Kayıplar listesindeki değerleri bir dağılım grafiği ile (scatterplot) çizdiriyoruz.
# alpha=0.1, noktaların biraz daha saydam olmasını sağlıyor ki yoğunluk anlaşılabilsin.

sns.scatterplot(x=range(len(losses)), y=losses, alpha=0.1)


# %% Modelin test aşaması
#--------------------------------------
# Eğitilmiş model ile test verisi (X_test) üzerinde tahmin yapıyoruz.
# torch.no_grad() => Bu blok içindeyken PyTorch, grad hesaplamalarını kapatır,
# yani tahmin sırasında ekstra hafıza kullanımı önlenir ve hız kazanılır.

X_test_torch = torch.FloatTensor(X_test)
with torch.no_grad():
    # Modelin çıktısını .round() ile 0 veya 1'e yuvarlıyoruz
    # Örneğin 0.7 -> 1, 0.3 -> 0 şeklinde
    y_test_hat = model(X_test_torch).round()


# %% Naive sınıflandırıcı (basit bir tahmin) doğruluğu
#-------------------------------------------------------
# Burada farklı bir bakış açısı olarak, "en sık rastlanan etiket kombinasyonunu" tahmin eden
# bir tür naive (basit) yaklaşımın doğruluk oranını hesaplıyoruz.
# y_test içindeki en sık görülen kombinasyon tespit edilir.
# Bu combinasyonun kaç defa görüldüğü (most_common_cnt) bulunur.
# Oradan bir yüzde doğruluk çıkar.

# Her satır (örneğin [1, 1, 0]) string'e dönüştürülüyor.
y_test_str = [str(i) for i in y_test.detach().numpy()]

# Sonra bu string'ler içinde en sık rastlananı buluyoruz.
most_common_cnt = Counter(y_test_str).most_common()[0][1]
print(f"Naive classifier: {most_common_cnt/len(y_test_str) * 100}%")


# %% Test doğruluğu hesaplama
#-----------------------------
# accuracy_score fonksiyonu y_test (gerçek) ve y_test_hat (tahmin) arasındaki
# tam eşleşme yüzdesini döndürür. Çoklu etiketli bir problemde
# "her bir etiket vektörünün aynı olup olmadığına" bakar.

test_acc = accuracy_score(y_test, y_test_hat)
print(f"Test accuracy: {test_acc * 100}%")


array([[1, 0, 1],
       [1, 0, 1],
       [0, 0, 0],
       [1, 0, 0],
       [1, 1, 0],
       [0, 0, 0],
       [0, 1, 0],
       [1, 0, 1],
       [1, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 0, 1],
       [1, 0, 1],
       [1, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 1],
       [1, 0, 0],
       [1, 1, 0],
       [1, 0, 1],
       [1, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [1, 0, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 1, 1],
       [1, 0, 1],
       [0, 0, 0],
       [1, 0, 1],
       [1, 1, 0],
       [1, 0, 1],
       [1, 0, 1],
       [1, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 1, 1],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 0, 1],
       [1, 0, 1],
       [1, 0, 1],
       [1, 0, 0],
       [1, 1, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0,